In [4]:
import torch
from torch import nn
from torchvision import models, transforms
import os
from pathlib import Path

In [9]:
# Find Data
data = Path("Data/")
image_path = data / "XRAY_DATA"

# Train/Test Directory
train_dir = image_path / "train"
test_dir = image_path / "test"